In [1]:
%load_ext autoreload

In [1]:
from certified_iris_generator import CertifiedIrisRegionGenerator
import sys
import os
import time
import numpy as np
from functools import partial
import itertools
import mcubes
import visualizations_utils as viz_utils
import iris_utils #TODO remove
from iris_plant_visualizer import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
from sandbox import rrtiris

In [2]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.all import InverseKinematics, RevoluteJoint, RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptionsRationalSpace, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
import pydrake.symbolic as sym
from pydrake.all import MathematicalProgram, RigidTransform, RollPitchYaw
import meshcat
from pydrake.all import GenerateSeedingPolytope, GenerateRandomSeedingPolytope
import pydrake.multibody.rational_forward_kinematics as rational_forward_kinematics
from pydrake.multibody.rational_forward_kinematics import FindEpsilonLower, FindEpsilonLowerVector, FindEpsilonUpperVector
from pydrake.solvers import mathematicalprogram as mp
from pydrake.all import MosekSolver, MosekSolverDetails, FindEpsTilCollisionOrRedundantForAllIneqs
from time import time

# Build Plant


In [3]:
q0 = [0.0, 0.0, 0.0]
q_low  = [-1.7, -2., -1.7]
q_high = [ 1.7,  2.,  1.7]


In [4]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
oneDOF_iiwa_asset = FindResourceOrThrow("drake/sos_iris_certifier/assets/oneDOF_iiwa7_with_box_collision.sdf")
twoDOF_iiwa_asset = FindResourceOrThrow("drake/sos_iris_certifier/assets/twoDOF_iiwa7_with_box_collision.sdf")

In [5]:
box_asset = FindResourceOrThrow("drake/sos_iris_certifier/assets/box_small.urdf")

models = []
models.append(parser.AddModelFromFile(box_asset))
models.append(parser.AddModelFromFile(twoDOF_iiwa_asset))
models.append(parser.AddModelFromFile(oneDOF_iiwa_asset))



locs = [[0.,0.,0.],[0.,.5,0.],[0.,-.5,0.]]
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("base", models[0]), RigidTransform(locs[0]))
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("iiwa_twoDOF_link_0", models[1]), RigidTransform(RollPitchYaw([0,0, -np.pi/2]).ToRotationMatrix(), locs[1]))
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("iiwa_oneDOF_link_0", models[2]), RigidTransform(RollPitchYaw([0,0, -np.pi/2]).ToRotationMatrix(), locs[2]))


plant.Finalize()

idx = 0
for model in models:
    for joint_index in plant.GetJointIndices(model):
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(q0[idx])
            joint.set_position_limits(lower_limits= np.array([q_low[idx]]), upper_limits= np.array([q_high[idx]]))
            idx += 1
        
            
#q_low = plant.GetPositionLowerLimits().tolist()
#q_high = plant.GetPositionUpperLimits().tolist()

Ratfk = RationalForwardKinematics(plant)


In [6]:
do_viz = True
visualizer = IrisPlantVisualizer(plant, builder, scene_graph, viz_role=Role.kIllustration)
diagram = visualizer.diagram
visualizer.visualize_collision_constraint(N = 30)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6001...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/
Connected to meshcat-server.


In [7]:
sliders = []
sliders.append(widgets.FloatSlider(min=q_low[0], max=q_high[0], value=0, description='q0'))
sliders.append(widgets.FloatSlider(min=q_low[1], max=q_high[1], value=0, description='q1'))
sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = q0.copy()
def handle_slider_change(change, idx):
    q[idx] = change['new']
    #print(q, end="\r")
    visualizer.showres(q)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)

visualizer.jupyter_cell()

Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


In [8]:
#filter fused joints self collisions 
digaram = visualizer.diagram
context = visualizer.diagram_context
plant_context = visualizer.plant_context
sg_context = scene_graph.GetMyContextFromRoot(context)
inspector = scene_graph.model_inspector()

pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()
print(len(inspector.GetCollisionCandidates()), "->", len(pairs))

gids = [gid for gid in inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)]
get_name_of_gid = lambda gid : inspector.GetName(gid)
gids.sort(key=get_name_of_gid)
iiwa_oneDOF_gids = [gid for gid in gids if "iiwa7_oneDOF::" in get_name_of_gid(gid)]
iiwa_twoDOF_gids = [gid for gid in gids if "iiwa7_twoDOF::" in get_name_of_gid(gid)]

oneDOF_fused_col_geom = iiwa_oneDOF_gids[2:]
iiwa_oneDOF_fused_set = GeometrySet(oneDOF_fused_col_geom)
twoDOF_fused_col_geom = iiwa_twoDOF_gids[4:]
iiwa_twoDOF_fused_set = GeometrySet(twoDOF_fused_col_geom)
# print([get_name_of_gid(gid) for gid in oneDOF_fused_col_geom])
# print([get_name_of_gid(gid) for gid in twoDOF_fused_col_geom])
scene_graph.collision_filter_manager()\
            .Apply(CollisionFilterDeclaration().ExcludeWithin(iiwa_oneDOF_fused_set))
scene_graph.collision_filter_manager()\
            .Apply(CollisionFilterDeclaration().ExcludeWithin(iiwa_twoDOF_fused_set))
pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()
print(len(inspector.GetCollisionCandidates()), "->", len(pairs))

150 -> 150
137 -> 150


# Setup IRIS Options and Generate Regions

In [9]:
# seed_points_q = np.array([[0.0, 0, 0], # zero config
#                         [0.0, -1.3, -1.3],  # start
#                         [0.9, -1.5, -0.8]     # goal
#                          ])
seed_points_q = np.array([[0.0, 0, 0], # zero config
                        [0.8, 1.3, -0.8],  # START: blue low green up
                        [0.1, 0.9, -1.2],     # GOAL: green low other up
                        [0.2, 1.6, -0.6],
                        [0.5, -1.9, -0.9],
                          [-0.6,-1.3,-0.3],
                          [-0.9,-1.7,-1.3],
                          [1.30, 1.50, -0.1],
                         ])[:,(0,1,2)]   # passing


seed_points = np.array([Ratfk.ComputeTValue(seed_points_q[idx], np.zeros((3,)))\
                        for idx in range(seed_points_q.shape[0])])
if do_viz:
    visualizer.plot_seedpoints(seed_points)

start = seed_points[1,:]
goal = seed_points[0,:]#seed_points[2,:]

#compute limits in t-space
limits_t = []
for q in [q_low, q_high]:
    limits_t.append(Ratfk.ComputeTValue(np.array(q), np.zeros((3,)) ))
    
starting_poly = HPolyhedron.MakeBox(limits_t[0], limits_t[1])
context = diagram.CreateDefaultContext()
q_star = np.zeros(3)
def check_empty_polytope(C, d):
    prog = MathematicalProgram()
    t = prog.NewContinuousVariables(C.shape[1], "t")

    prog.AddLinearConstraint(C,-np.inf*np.ones(C.shape[0]), d, t)
    prog.AddBoundingBoxConstraint(limits_t[0], limits_t[1], t)
    result = MosekSolver().Solve(prog)
    return result.is_success(), result.get_solution_result()

In [11]:
# RUN THIS CELL IF YOU WANT TO DO SNOPT IRIS

# regions = []

# iris_options = IrisOptionsRationalSpace()
# iris_options.require_sample_point_is_contained = True
# iris_options.iteration_limit = 20
# iris_options.configuration_space_margin = 1e-5
# iris_options.max_faces_per_collision_pair = 60
# iris_options.termination_threshold = -1
# iris_options.q_star = np.zeros(3)
# iris_options.relative_termination_threshold = 0.05
# iris_options.enable_ibex = False
# #deprecated
# iris_options.certify_region_with_sos_during_generation = False
# iris_options.certify_region_with_sos_after_generation = False

# for i, s in enumerate(seed_points):
#     plant.SetPositions(plant.GetMyMutableContextFromRoot(context), s)
#     if False:
#         #starting_hpolyhedron = regions[i-1]
#         r = IrisInRationalConfigurationSpace (plant, plant.GetMyContextFromRoot(context),
#                                               iris_options, starting_hpolyhedron)
#     else:
#         r = IrisInRationalConfigurationSpace(plant, plant.GetMyContextFromRoot(context), iris_options)
#     regions.append(r)
#     print(f'Completed region: {i+1}/{len(seed_points)}')
#     print(f"Sample point contained = {np.all(r.A()@s <= r.b())}")
#     print(f"Sample point contained componentwise = {r.A()@s <= r.b()}")
#     print()

In [12]:
# if do_viz:
#     visualizer.plot_regions(regions, ellipses=None, region_suffix='original')

## Create default region

In [13]:
regions_default = []
max_vects = []
min_vects = []
vector_bisection_options_list = []
bilinear_alteration_options_list = []
interleaved_options_list = []

#parameters for creating a default region. Currently set to create a box
# num_perm_dim = 0
# num_rot = 1

num_unit_box_copies = 2
initial_box_scale = 0.25
gaussian_variance = 0.1

for i, s in enumerate(seed_points):
#     regions_default.append(GenerateSeedingPolytope(s, num_perm_dim, num_rot))
    regions_default.append(GenerateRandomSeedingPolytope(s,
                                     num_unit_box_copies,
                                     initial_box_scale,
                                     gaussian_variance))
    eps_min_vect = FindEpsilonLowerVector(regions_default[i].A(), 
                                      regions_default[i].b(), 
                                      limits_t[0], limits_t[1], s)
    
    min_vects.append(eps_min_vect)
    
#     eps_max_vect = FindEpsTilCollisionOrRedundantForAllIneqs(plant,
# #                                                              context,
#                                                              plant.GetMyMutableContextFromRoot(context), 
#                                 q_star, regions_default[i].A(), regions_default[i].b(), 
# #                                          eps_min_vect, 
#                                                              limits_t[0], limits_t[1], s)
    eps_max_vect = 10*regions_default[i].b()
    max_vects.append(eps_max_vect)
    
    vector_bisection_search_options = rational_forward_kinematics.VectorBisectionSearchOption()
    vector_bisection_search_options.max_iters = 10
    vector_bisection_search_options.max_feasible_iters = 5
    vector_bisection_search_options.search_d = True
    vector_bisection_search_options.epsilon_min = np.zeros_like(regions_default[i].b())
    vector_bisection_search_options.epsilon_max = eps_max_vect
    vector_bisection_search_options.num_threads = -1
    vector_bisection_options_list.append(vector_bisection_search_options)
    
    bilinear_alternation_options = rational_forward_kinematics.BilinearAlternationOption()
    bilinear_alternation_options.max_iters = 20
    bilinear_alternation_options.lagrangian_backoff_scale = 0
    bilinear_alternation_options.polytope_backoff_scale = 0
    bilinear_alternation_options.num_threads = -1
    bilinear_alteration_options_list.append(bilinear_alternation_options)
    
    binary_search_option = rational_forward_kinematics.BinarySearchOption()
    binary_search_option.epsilon_max = 5
    binary_search_option.epsilon_min = FindEpsilonLower(regions_default[i].A(), 
                                                        regions_default[i].b()+eps_min_vect, 
                                                        limits_t[0], limits_t[1], s)
    binary_search_option.max_iters = 10
    binary_search_option.search_d = True
    binary_search_option.num_threads = 4
    
    interleaved_options = rational_forward_kinematics.InterleavedRegionSearchOptions()
    interleaved_options.scalar_binary_search_options = binary_search_option
    interleaved_options.vector_bisection_search_options = vector_bisection_search_options
    interleaved_options.bilinear_alternation_options = bilinear_alternation_options
    #actually does round robin
    interleaved_options.max_method_switch = 3
    interleaved_options_list.append(interleaved_options)
    
    
num_round_robin_rounds = 1

if do_viz:
    visualizer.plot_regions(regions_default, ellipses=None, region_suffix='_default')
    

In [14]:
# i = 1
# s = seed_points[i,:]
# C, d = regions_default[i].A(), regions_default[i].b()

# eps_min_vect = FindEpsilonLowerVector(regions_default[i].A(), 
#                                       regions_default[i].b(), 
#                                       limits_t[0], limits_t[1], s)
# # eps_max_vect = FindEpsTilCollisionOrRedundantForAllIneqs(plant,plant.GetMyMutableContextFromRoot(context), 
# #                                 q_star, regions_default[i].A(), regions_default[i].b()+eps_min_vect, 
# #                                                     limits_t[0], limits_t[1], s)


# binary_search_option = rational_forward_kinematics.BinarySearchOption()
# binary_search_option.epsilon_max = 5
# binary_search_option.epsilon_min = FindEpsilonLower(C, d+eps_min_vect, limits_t[0], limits_t[1], s)
# binary_search_option.max_iters = 10
# binary_search_option.search_d = True
# binary_search_option.num_threads = -1

# vector_bisection_search_options = rational_forward_kinematics.VectorBisectionSearchOption()
# vector_bisection_search_options.max_iters = 10
# vector_bisection_search_options.max_feasible_iters = 5
# vector_bisection_search_options.search_d = True
# vector_bisection_search_options.epsilon_min = np.zeros_like(d)
# vector_bisection_search_options.epsilon_max = 5*np.abs(eps_min_vect)
# vector_bisection_search_options.num_threads = -1
# vector_bisection_options_list.append(vector_bisection_search_options)

# bilinear_alternation_options = rational_forward_kinematics.BilinearAlternationOption()
# bilinear_alternation_options.max_iters = 5
# bilinear_alternation_options.lagrangian_backoff_scale = 0
# bilinear_alternation_options.polytope_backoff_scale = 0
# bilinear_alternation_options.num_threads = -1
# bilinear_alteration_options_list.append(bilinear_alternation_options)

# interleaved_options = rational_forward_kinematics.InterleavedRegionSearchOptions()
# interleaved_options.scalar_binary_search_options = binary_search_option
# interleaved_options.vector_bisection_search_options = vector_bisection_search_options
# interleaved_options.bilinear_alternation_options = bilinear_alternation_options
# #actually does round robin
# interleaved_options.max_method_switch = 1

# # vector_bisection_search_options = vector_bisection_options_list[i]
# # interleaved_options = interleaved_options_list[i]
# # r_moved_list = []
# # for j in range(len(d)):
# #     d_new = d.copy()
# #     d_new[j] += eps_max_vect[j]
# #     r_moved_list.append(HPolyhedron(C, d_new))
# # if do_viz:
# #     visualizer.plot_regions(r_moved_list, ellipses=None, region_suffix=f'_default_{i}_maxed')
# # if do_viz:
# #     visualizer.plot_regions([HPolyhedron(C, d+0.99*eps_max_vect)], ellipses=None, region_suffix=f'_all_{i}_maxed')

In [15]:
# #plot the new polytopes which each individual face moved
# if do_viz:
#     for i, r in enumerate(regions_default):
#         C, d = r.A(), r.b()
#         eps_max = vector_bisection_options_list[i].epsilon_max
#         r_moved_list = []
#         for j in range(len(d)):
#             d_new = d.copy()
#             d_new[j] += eps_max[j]
#             r_moved_list.append(HPolyhedron(C, d_new))
#         if do_viz:
#             visualizer.plot_regions(r_moved_list, ellipses=None, region_suffix=f'_default_{i}_maxed')
#         if do_viz:
#             visualizer.plot_regions([HPolyhedron(C, d+0.99*eps_max)], ellipses=None, region_suffix=f'_all_{i}_maxed')

## Run Cspace Free Region

In [16]:
cspace_free_region = rational_forward_kinematics.CspaceFreeRegion(diagram, plant, scene_graph,
                                   rational_forward_kinematics.SeparatingPlaneOrder.kAffine,
                                   rational_forward_kinematics.CspaceRegionType.kGenericPolytope)
filtered_collision_pairs = set()
solver_options = mp.SolverOptions()
# solver_options.SetOption(mp.CommonSolverOption.kPrintToConsole, 1)

In [17]:
# cspace_free_region_solution_binary_search = cspace_free_region.CspacePolytopeBinarySearch(q_star,
#                                                                      filtered_collision_pairs,
#                                                                      C, d+eps_min_vect, binary_search_option, 
#                                                                      solver_options, s)

In [18]:
# C, d = cspace_free_region_solution_binary_search.C, cspace_free_region_solution_binary_search.d
# interleaved_options.vector_bisection_search_options.epsilon_min = np.zeros(d.shape)
# r = HPolyhedron(np.vstack([starting_poly.A(),cspace_free_region_solution_binary_search.C]),
#                 np.hstack([starting_poly.b(), cspace_free_region_solution_binary_search.d])
#                 )
# e = Hyperellipsoid(np.linalg.inv(cspace_free_region_solution_binary_search.P), 
#                    cspace_free_region_solution_binary_search.q)
# if do_viz:
#     visualizer.plot_regions([r], 
#                             ellipses=[e], region_suffix='_binary_search_single')

In [19]:
# t0 = time()
# cspace_free_region_solution_interleaved = cspace_free_region.InterleavedCSpacePolytopeSearch(q_star,
#                                                                      filtered_collision_pairs,
#                                                                      C, d, num_round_robin_rounds, 
#                                                                      interleaved_options,
#                                                                      solver_options, s, 
#                                                                      context = plant.GetMyMutableContextFromRoot(context))
# t1 = time()


In [20]:
# print(f"Completed points in {t1-t0}s")

In [21]:
# r = HPolyhedron(np.vstack([starting_poly.A(),cspace_free_region_solution_interleaved.C]),
#                 np.hstack([starting_poly.b(), cspace_free_region_solution_interleaved.d])
#                 )
# # e = Hyperellipsoid(np.linalg.inv(cspace_free_region_solution_interleaved.P), 
# #                    cspace_free_region_solution_interleaved.q)

# if do_viz:
#     visualizer.plot_regions([r], 
#                             ellipses=None, region_suffix='_interleaved_single')

In [22]:
# check_empty_polytope(r.A(), r.b())

In [23]:
# r = HPolyhedron(np.vstack([starting_poly.A(),cspace_free_region_solution_round_robin.C]),
#                 np.hstack([starting_poly.b(), cspace_free_region_solution_round_robin.d])
#                 )
# e = Hyperellipsoid(np.linalg.inv(cspace_free_region_solution_round_robin.P), 
#                    cspace_free_region_solution_round_robin.q)
# if do_viz:
#     visualizer.plot_regions([r], 
#                             ellipses=[e], region_suffix='_round_robin_single')

In [24]:
seed_point_list = [s for s in seed_points] 
print(len(seed_point_list))
C_mats = [r.A() for r in regions_default]
d_vects = [r.b() for r in regions_default]
t0 = time()
cspace_free_region_solution_interleaved_vect = cspace_free_region.InterleavedCSpacePolytopeSearchForSeedPoints(
                                                                    q_star,
                                                                     filtered_collision_pairs,
                                                                     C_mats, d_vects, num_round_robin_rounds, 
                                                                     interleaved_options_list,
                                                                     solver_options, seed_point_list, 
    context = plant.GetMyMutableContextFromRoot(context))
# cspace_free_region_solution_round_robin_vect = cspace_free_region.CspacePolytopeRoundRobinBisectionSearchForSeedPoints(
#                                                                     q_star,
#                                                                      filtered_collision_pairs,
#                                                                      C_mats, d_vects, num_round_robin_rounds, 
#                                                                      vector_bisection_options_list,
#                                                                      solver_options, seed_point_list)
t1 = time()


[2022-02-09 19:32:03.443] [console] [info] starting binary search
[2022-02-09 19:32:03.443] [console] [info] starting binary search
[2022-02-09 19:32:03.443] [console] [info] starting binary search
[2022-02-09 19:32:03.443] [console] [info] starting binary search
[2022-02-09 19:32:03.443] [console] [info] starting binary search
[2022-02-09 19:32:03.443] [console] [info] starting binary search
[2022-02-09 19:32:03.443] [console] [info] starting binary search
[2022-02-09 19:32:03.443] [console] [info] starting binary search
[2022-02-09 19:32:04.908] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(box_scene::box1_Collision@0000000003c098a8, iiwa7_twoDOF::iiwa_twoDOF_link_2_collision)
(box_scene::box1_Collision@0000000003c098a8, iiwa7_twoDOF::iiwa_twoDOF_link_4_collision)
(box_scene::box1_Collision@0000000003c098a8, iiwa7_twoDOF::iiwa_twoDOF_link_5_collision)



8


[2022-02-09 19:32:04.926] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(box_scene::box1_Collision@0000000003c098a8, iiwa7_twoDOF::iiwa_twoDOF_link_2_collision)
(box_scene::box1_Collision@0000000003c098a8, iiwa7_twoDOF::iiwa_twoDOF_link_4_collision)
(box_scene::box1_Collision@0000000003c098a8, iiwa7_twoDOF::iiwa_twoDOF_link_5_collision)

[2022-02-09 19:32:04.951] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(box_scene::box1_Collision@0000000003c098a8, iiwa7_twoDOF::iiwa_twoDOF_link_2_collision)
(box_scene::box1_Collision@0000000003c098a8, iiwa7_twoDOF::iiwa_twoDOF_link_4_collision)
(box_scene::box1_Collision@0000000003c098a8, iiwa7_twoDOF::iiwa_twoDOF_link_5_collision)

[2022-02-09 19:32:04.977] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(box_scene::box1_Collision@0000000003c098a8, iiwa7_twoDOF::iiwa_twoDOF_link_2_collision)
(box_scene::box1_Collision@0000000003c098a8, iiwa7_two

[2022-02-09 19:32:36.751] [console] [info] max(power(det(P), 1/4))=1.3518785152397902e-06, solver_time 0.01599407196044922
[2022-02-09 19:32:36.839] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(box_scene::box1_Collision@0000000003c098a8, iiwa7_twoDOF::iiwa_twoDOF_link_2_collision)
(box_scene::box1_Collision@0000000003c098a8, iiwa7_twoDOF::iiwa_twoDOF_link_4_collision)
(box_scene::box1_Collision@0000000003c098a8, iiwa7_twoDOF::iiwa_twoDOF_link_5_collision)

[2022-02-09 19:32:36.841] [console] [info] epsilon=1.2500000075 is infeasible
[2022-02-09 19:32:36.902] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(box_scene::box1_Collision@0000000003c098a8, iiwa7_twoDOF::iiwa_twoDOF_link_2_collision)
(box_scene::box1_Collision@0000000003c098a8, iiwa7_twoDOF::iiwa_twoDOF_link_5_collision)

[2022-02-09 19:32:36.903] [console] [info] epsilon=1.2500000075 is infeasible
[2022-02-09 19:32:37.526] [console] [warning] Cannot find La

[2022-02-09 19:32:42.111] [console] [info] epsilon=0.31250001299624064 is infeasible
[2022-02-09 19:32:42.189] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(box_scene::box2_Collision@0000000003c09d58, iiwa7_twoDOF::iiwa_twoDOF_link_4_collision)

[2022-02-09 19:32:42.196] [console] [info] epsilon=0.312500009375 is infeasible
[2022-02-09 19:32:43.379] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(box_scene::box2_Collision@0000000003c09d58, iiwa7_twoDOF::iiwa_twoDOF_link_4_collision)
(box_scene::box2_Collision@0000000003c09d58, iiwa7_twoDOF::iiwa_twoDOF_link_5_collision)
(box_scene::box2_Collision@0000000003c09d58, iiwa7_twoDOF::iiwa_twoDOF_link_6_collision)

[2022-02-09 19:32:43.382] [console] [info] epsilon=0.312500009375 is infeasible
[2022-02-09 19:32:43.773] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(box_scene::box2_Collision@0000000003c09d58, iiwa7_twoDOF::iiwa_twoDOF_link_

...

[2022-02-09 20:38:14.920] [console] [info] current gap is epsilon_gap=0.16260366064453358
[2022-02-09 20:38:14.921] [console] [info] current gap is epsilon_gap=0.16260366064453358
[2022-02-09 20:38:18.796] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 20:38:18.994] [console] [info] bilinear alt on d succeeded
[2022-02-09 20:38:19.016] [console] [info] epsilon=0.04251134964071829 is feasible
[2022-02-09 20:38:21.048] [console] [info] bilinear alt on d succeeded
[2022-02-09 20:38:21.070] [console] [info] epsilon=0.0813018306790727 is feasible
[2022-02-09 20:38:28.341] [console] [info] reset eps_min=0.04267829377677157, eps_max=0.04682220661645736
[2022-02-09 20:38:28.341] [console] [info] current gap is epsilon_gap=0.004143912839685793
[2022-02-09 20:38:32.191] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 20:38:34.214] [console] [info] reset eps_min=0.08130187537245526, eps_max=0.1626036605516763
[2022-02-09 20:38:34.215]

[2022-02-09 20:41:09.441] [console] [info] epsilon=0.16244499647934219 is feasible
[2022-02-09 20:41:16.561] [console] [info] reset eps_min=0.005283323688566932, eps_max=0.006952640162585094
[2022-02-09 20:41:16.562] [console] [info] current gap is epsilon_gap=0.001669316474018162
[2022-02-09 20:41:20.416] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 20:41:22.475] [console] [info] reset eps_min=0.16244503351905437, eps_max=0.1626038014884903
[2022-02-09 20:41:22.476] [console] [info] Ineq 2/12 in round 1
[2022-02-09 20:41:23.010] [console] [info] bilinear alt on d succeeded
[2022-02-09 20:41:23.032] [console] [info] epsilon=0.006117981925576013 is feasible
[2022-02-09 20:41:30.366] [console] [info] current gap is epsilon_gap=0.0
[2022-02-09 20:41:30.366] [console] [info] Ineq 3/12 in round 1
[2022-02-09 20:41:33.236] [console] [info] reset eps_min=0.006118237200187671, eps_max=0.006925389101791496
[2022-02-09 20:41:33.237] [console] [info] current gap 

[2022-02-09 20:44:13.944] [console] [info] bilinear alt on d succeeded
[2022-02-09 20:44:13.966] [console] [info] epsilon=0.011109117184207637 is feasible
[2022-02-09 20:44:16.401] [console] [info] reset eps_min=0.09693861949264332, eps_max=0.09731813118389437
[2022-02-09 20:44:16.402] [console] [info] current gap is epsilon_gap=0.00037951169125105544
[2022-02-09 20:44:20.330] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 20:44:22.716] [console] [info] bilinear alt on d succeeded
[2022-02-09 20:44:22.739] [console] [info] epsilon=0.09712837533826885 is feasible
[2022-02-09 20:44:25.920] [console] [info] reset eps_min=0.01110915119009978, eps_max=0.011467153502688954
[2022-02-09 20:44:25.920] [console] [info] current gap is epsilon_gap=0.0003580023125891729
[2022-02-09 20:44:29.830] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 20:44:32.324] [console] [info] bilinear alt on d succeeded
[2022-02-09 20:44:32.346] [console] 

[2022-02-09 20:47:11.599] [console] [info] Lagrangian step time 3.894 s
[2022-02-09 20:47:11.604] [console] [info] max(power(det(P), 1/4))=0.4872476972309925, solver_time 0.002624034881591797
[2022-02-09 20:47:11.604] [console] [info] cost improvement inf
[2022-02-09 20:47:14.255] [console] [info] Iter: 0, polytope step cost -0.16290428830224743, solver time 0.5812859535217285
[2022-02-09 20:47:18.150] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 20:47:18.152] [console] [info] Lagrangian step time 3.874 s
[2022-02-09 20:47:18.167] [console] [info] max(power(det(P), 1/4))=0.4913670248727841, solver_time 0.009495973587036133
[2022-02-09 20:47:18.167] [console] [info] cost improvement 0.004119327641791626
[2022-02-09 20:47:19.907] [console] [info] reset eps_min=0.05077656845333256, eps_max=0.05097567819486404
[2022-02-09 20:47:19.908] [console] [info] current gap is epsilon_gap=0.00019910974153147798
[2022-02-09 20:47:23.779] [console] [info] Found Lagran

[2022-02-09 20:49:36.140] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 20:49:38.443] [console] [info] bilinear alt on d succeeded
[2022-02-09 20:49:38.465] [console] [info] epsilon=0.021713990465716176 is feasible
[2022-02-09 20:49:48.871] [console] [info] reset eps_min=0.0217139902918958, eps_max=0.023161347473855176
[2022-02-09 20:49:48.871] [console] [info] current gap is epsilon_gap=0.0014473571819593745
[2022-02-09 20:49:52.775] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 20:49:55.107] [console] [info] bilinear alt on d succeeded
[2022-02-09 20:49:55.130] [console] [info] epsilon=0.02243766888287549 is feasible
[2022-02-09 20:50:05.507] [console] [info] reset eps_min=0.02243766885332793, eps_max=0.023161337364471235
[2022-02-09 20:50:05.507] [console] [info] current gap is epsilon_gap=0.000723668511143305
[2022-02-09 20:50:09.442] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 20:5

In [25]:
print(seed_points)

[[ 0.          0.          0.        ]
 [ 0.42279322  0.7602044  -0.42279322]
 [ 0.05004171  0.48305507 -0.68413681]
 [ 0.10033467  1.02963856 -0.30933625]
 [ 0.25534192 -1.39838259 -0.48305507]
 [-0.30933625 -0.7602044  -0.15113522]
 [-0.48305507 -1.13833271 -0.7602044 ]
 [ 0.7602044   0.93159646 -0.05004171]]


In [26]:
print(f"Completed points in {t1-t0}s")
regions_round_robin = []
ellipses_round_robin = []
for sol in cspace_free_region_solution_interleaved_vect:
    r = HPolyhedron(np.vstack([starting_poly.A(),sol.C]),
                np.hstack([starting_poly.b(), sol.d])
                )
    e = Hyperellipsoid(np.linalg.inv(sol.P), sol.q)

    regions_round_robin.append(r)
    ellipses_round_robin.append(e)
if do_viz:
    visualizer.plot_regions(regions_round_robin, 
                            ellipses=None, region_suffix='_interleaved2')

Completed points in 4891.462082147598s


In [62]:
print(f"Completed points in {t1-t0}s")
regions_round_robin = []
ellipses_round_robin = []
for i, sol in enumerate(cspace_free_region_solution_interleaved_vect):
    dtmp = sol.d+sol.C@seed_points[i,:]
    
    r = HPolyhedron(np.vstack([starting_poly.A(),sol.C*2]),
                np.hstack([starting_poly.b(), dtmp])
                )
    e = Hyperellipsoid(np.linalg.inv(sol.P), sol.q)

    regions_round_robin.append(r)
    ellipses_round_robin.append(e)
if do_viz:
    visualizer.plot_regions(regions_round_robin, 
                            ellipses=None, region_suffix='_interleaved_small')

Completed points in 4891.462082147598s


In [28]:
from datetime import datetime

# datetime object containing current date and time
now = datetime.now()
 
print("now =", now)

# dd/mm/YY H:M:S
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("finished running at =", dt_string)	

now = 2022-02-09 21:08:40.719229
finished running at = 09/02/2022 21:08:40


In [27]:
visualizer.vis2["regions_interleaved"]

<Visualizer using: <meshcat.visualizer.ViewerWindow object at 0x7f2eafc67a60> at path: <meshcat.path.Path object at 0x7f2eafc3cd30>>

In [46]:
N = 50000
collision_free_points_t = np.zeros((N,3))
col_func_handle_t = partial(visualizer.eval_cons, c=visualizer.collision_constraint, tol=0.01)
col_func_handle_t(Ratfk.ComputeTValue(t, np.array([0,0,0])))

#sample N collision free points
for idx in range(N):
        if idx%400 ==0:
            print(idx)
        rand = np.random.rand(3)
        t = limits_t[0] + rand*(limits_t[1]-limits_t[0])
        while col_func_handle_t(t):
            rand = np.random.rand(3)
            t = limits_t[0] + rand*(limits_t[1]-limits_t[0])
        collision_free_points_t[idx, :] = t

#check containment
num_contained_in_regions = 0
for pt in collision_free_points_t:
    for region in regions_round_robin:
        if region.PointInSet(pt):
            num_contained_in_regions +=1
            break
        
print("Num col free points: ", N, "Num contained in round robin regions: ", num_contained_in_regions)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [33]:
visualizer.q_star = 0.0*np.zeros((3,1))
print(visualizer.q_star.dtype)
print(t.dtype)
visualizer.col_func_handle_rational(t.reshape(-1,1))

float64
float64


TypeError: ComputeQValue(): incompatible function arguments. The following argument types are supported:
    1. (self: pydrake.multibody.rational_forward_kinematics.RationalForwardKinematics, t_val: numpy.ndarray[numpy.float64[m, 1]], q_star_val: numpy.ndarray[numpy.float64[m, 1]]) -> numpy.ndarray[numpy.float64[m, 1]]

Invoked with: <pydrake.multibody.rational_forward_kinematics.RationalForwardKinematics object at 0x7f3a726ae230>, array([[[ 0.43109738],
        [-0.84842998],
        [ 0.96040215]]]), array([[0.],
       [0.],
       [0.]])

In [11]:
rand = np.random.rand(3)
pos_samp = limits_t[0] + rand*(limits_t[1]-limits_t[0])


1.0

# Don't run after here

In [36]:
cspace_free_region_solution_round_robin = cspace_free_region.CspacePolytopeRoundRobinBisectionSearch(q_star,
                                                                     filtered_collision_pairs,
                                                                     C, d, 1, 
                                                                     vector_bisection_search_options,
                                                                     solver_options, s)

[2022-02-09 11:46:31.589] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 11:46:34.425] [console] [info] bilinear alt on d succeeded
[2022-02-09 11:46:34.475] [console] [info] Ineq 1/12 in round 1
[2022-02-09 11:46:34.476] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-09 11:46:34.491] [console] [info] output of eps_redundant_max is -3.436515294552578e-09
[2022-02-09 11:46:34.492] [console] [warning] round robin bisection search failed: epsilon max is less than epsilon min.
 difference is = 
     -0.25
-0.238571
-0.225191
 -0.20519
    -0.25
    -0.25
-0.239698
    -0.25
    -0.25
    -0.25
    -0.25
    -0.25
[2022-02-09 11:46:34.493] [console] [info] Ineq 2/12 in round 1
[2022-02-09 11:46:34.511] [console] [info] output of eps_redundant_max is 6.756142090225126e-09
[2022-02-09 11:46:34.512] [console] [info] Ineq 3/12 in round 1
[2022-02-09 11:46:34.529] [console] [info] output of eps_redund

[2022-02-09 11:47:44.971] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 11:47:47.809] [console] [info] bilinear alt on d succeeded
[2022-02-09 11:47:47.841] [console] [info] epsilon=-0.1569430869108956 is feasible
[2022-02-09 11:47:47.842] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-09 11:47:47.851] [console] [info] output of eps_redundant_max is 0.0007903398544878426
[2022-02-09 11:47:47.852] [console] [info] reset eps_min=-0.15694308925800982, eps_max=-0.15615274940352197
[2022-02-09 11:47:47.853] [console] [info] current gap is epsilon_gap=0.0007903398544878426
[2022-02-09 11:47:53.058] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 11:47:55.860] [console] [info] bilinear alt on d succeeded
[2022-02-09 11:47:55.892] [console] [info] epsilon=-0.1565479193307659 is feasible
[2022-02-09 11:47:55.893] [console] [info] epsilon exceeds components of epsilon m

In [29]:
cspace_free_region_solution_bilinear = cspace_free_region.CspacePolytopeBilinearAlternation(q_star,
                                                                     filtered_collision_pairs,
                                                                     cspace_free_region_solution_round_robin.C,
                                                                    0.75*cspace_free_region_solution_round_robin.d,
                                                                     bilinear_alternation_options,
                                                                     solver_options, s)

[2022-02-08 19:34:36.721] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 19:34:36.723] [console] [info] Lagrangian step time 3.045 s
[2022-02-08 19:34:36.751] [console] [info] max(power(det(P), 1/4))=0.2071381570004865, solver_time 0.018611907958984375
[2022-02-08 19:34:36.753] [console] [info] cost improvement inf
[2022-02-08 19:34:38.615] [console] [info] Iter: 0, polytope step cost -0.008828768121858371, solver time 0.48531293869018555
[2022-02-08 19:34:41.859] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 19:34:41.860] [console] [info] Lagrangian step time 3.218 s
[2022-02-08 19:34:41.881] [console] [info] max(power(det(P), 1/4))=0.2105033555416147, solver_time 0.013540029525756836
[2022-02-08 19:34:41.882] [console] [info] cost improvement 0.0033651985411282115
[2022-02-08 19:34:43.780] [console] [info] Iter: 1, polytope step cost -0.008522369595631428, solver time 0.532606840133667
[2022-02-08 19:34:46.920] [console

[2022-02-08 19:36:00.640] [console] [info] Iter: 16, polytope step cost -0.00557567747292513, solver time 0.38878607749938965
[2022-02-08 19:36:04.137] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 19:36:04.141] [console] [info] Lagrangian step time 3.472 s
[2022-02-08 19:36:04.172] [console] [info] max(power(det(P), 1/4))=0.24350116763464993, solver_time 0.01593494415283203
[2022-02-08 19:36:04.174] [console] [info] cost improvement 0.001400599503426303
[2022-02-08 19:36:05.966] [console] [info] Iter: 17, polytope step cost -0.005466929346269455, solver time 0.3860509395599365
[2022-02-08 19:36:09.113] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 19:36:09.114] [console] [info] Lagrangian step time 3.119 s
[2022-02-08 19:36:09.142] [console] [info] max(power(det(P), 1/4))=0.24485582644352566, solver_time 0.00913095474243164
[2022-02-08 19:36:09.145] [console] [info] cost improvement 0.001354658808875725
[2022-02-08 19:3

In [31]:
if do_viz:
    visualizer.plot_regions([HPolyhedron(cspace_free_region_solution_bilinear.C,
                                         cspace_free_region_solution_bilinear.d)], 
                            ellipses=None, region_suffix='_bilinear')

In [15]:
regions_round_robin = []
ellipses_round_robin = []
for sol in cspace_free_region_solution_round_robin_vect:
    r = HPolyhedron(np.vstack([starting_poly.A(),sol.C]),
                np.hstack([starting_poly.b(), sol.d])
                )
    e = Hyperellipsoid(np.linalg.inv(sol.P), sol.q)

    regions_round_robin.append(r)
    ellipses_round_robin.append(e)
if do_viz:
    visualizer.plot_regions(regions_round_robin, 
                            ellipses=ellipses_round_robin, region_suffix='_round_robin')

In [22]:
if do_viz:
    visualizer.plot_regions([HPolyhedron(cspace_free_region_solution.C, 0.5*cspace_free_region_solution.d)], 
                            ellipses=None, region_suffix='_round_robin')

In [26]:
cspace_free_region_solution_vector_bisection = cspace_free_region.CspacePolytopeBisectionSearchVector(q_star,
                                                                     filtered_collision_pairs,
                                                                     C, d, 
                                                                     vector_bisection_search_options,
                                                                     solver_options, s)

[2022-02-07 15:27:37.125] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-07 15:27:37.194] [console] [info] max(power(det(P), 1/4))=-nan, solver_time 0.05913114547729492
[2022-02-07 15:27:40.385] [console] [info] bilinear alt on d succeeded
[2022-02-07 15:27:40.411] [console] [info] max(power(det(P), 1/4))=1.195363453968912e-06, solver_time 0.014796972274780273
[2022-02-07 15:27:45.796] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(iiwa7_twoDOF::iiwa_twoDOF_link_3_collision, iiwa7_oneDOF::iiwa_oneDOF_link_7_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_4_collision, iiwa7_oneDOF::iiwa_oneDOF_link_6_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_4_collision, iiwa7_oneDOF::iiwa_oneDOF_link_7_collision)

[2022-02-07 15:27:45.797] [console] [info] total iter=1, feasible iter = 0
[2022-02-07 15:27:51.001] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(iiwa7_twoDOF::iiwa_twoDOF_link_4_collision, iiwa7_

In [29]:
if do_viz:
    visualizer.plot_regions([HPolyhedron(cspace_free_region_solution_vector_bisection.C, 
                                         cspace_free_region_solution_vector_bisection.d)], 
                            ellipses=None, region_suffix='_vector_bisection')

(True, <SolutionResult.kSolutionFound: 0>)


In [96]:
i

2